In [1]:
import numpy as np
import scipy as sp
import sys
from mendeleev import element
import mdtraj as md
import os

In [2]:
%load_ext autoreload
sys.path.append("/home/gmancini/Dropbox/appunti/old_search_algos_28_08_2020/molecule_utils")
import zmatrix, quaternion_utils
%autoreload 2

In [3]:
har2kjmol = sp.constants.physical_constants["Avogadro constant"][0]\
*sp.constants.physical_constants["Hartree energy"][0]/1000.
har2kjmol

2625.4996394798254

In [4]:
os.chdir("/home/gmancini/data/devel/gaussian/j14/working")

In [5]:
covalent_radii = dict(zip(("C","N","O"),\
                     [0.01*(lambda x: element(x).covalent_radius_bragg)(i) for i in ("C","N","O")]))
covalent_radii['H'] = 0.01*element('H').covalent_radius_pyykko
covalent_radii

{'C': 0.77, 'N': 0.65, 'O': 0.65, 'H': 0.32}

In [6]:
zeff = dict(zip(("C","H","N","O"),[(lambda x: element(x).zeff(o='s')-0.4)(i) for i in ("C","H","N","O")]))
zeff

{'C': 2.85, 'H': 0.6, 'N': 3.5000000000000004, 'O': 4.1499999999999995}

In [7]:
#elec = dict(zip(("C","H","N","O"),\
#                     [(lambda x: element(x).electronegativity(scale='allred-rochow'))\
#                     (i) for i in ("C","H","N","O")]))
# allrd-rochov scale is not in the expected values or units
# quick & dirty estimate from eq. and adjust zeff as above
elec =dict(zip(("C","H","N","O"),(2.5,2.2,3.07,3.50)))
elec

{'C': 2.5, 'H': 2.2, 'N': 3.07, 'O': 3.5}

In [8]:
tol = 0.6

In [9]:
tempo = md.load("tempo.pdb")
tempo_top = tempo.topology
tempo_atoms = [a.element.symbol for a in tempo_top.atoms]

In [10]:
water = md.load("tip3p_fb_3351.pdb")
water_top = water.topology
water_atoms = [a.element.symbol for a in water_top.atoms]

In [11]:
water_top.select("resid 1")

array([3, 4, 5])

In [12]:
def create_hole(solute, solvent, rsphere, radii, elec, tol, outname):
    """
    create hole using covalent radii and same eq as proxima with
    role of tolerance reversed
    """
    solute_top = solute.topology
    solvent_top = solvent.topology
    solute_atoms = [a.element.symbol for a in solute_top.atoms]
    solvent_atoms = np.asarray([a.element.symbol for a in solvent_top.atoms])
    solvent_natoms = len(solvent_top.select("resid 1"))
    residues = list(range(solvent_top.n_residues))
    remove = list()
    rsphere = rsphere/10.
    solvent.xyz[0] = solvent.xyz[0] - rsphere
    for res in range(len(residues)):
        ratoms = solvent_top.select("resid " + str(res))
        if len(ratoms) == 0: 
            continue
        for jatom in range(solute.n_atoms):
            jelem = solute_atoms[jatom]
            if jelem == "VS": continue
            D = 10.*np.linalg.norm(solvent.xyz[0][ratoms] - solute.xyz[0][jatom], axis=1)
            C = np.array([(radii[i] + radii[jelem]-0.07*(elec[i]-elec[jelem])**2 + tol)\
                           for i in solvent_atoms[ratoms]])
            #print(solvent.xyz[0][ratoms],D,C,solute.xyz[0][jatom])
            if np.min(D) <= np.max(C):
                remove.append(res)
            #break
        #break
    print(remove)
    okres = list(set(residues).difference(remove))
    rlist = "resid " + ' '.join(list(map(str,okres)))
    solvent.restrict_atoms(solvent_top.select(rlist))
    solvent.save(outname)
    return okres

In [13]:
hole = create_hole(tempo, water, 20., covalent_radii, elec, tol, "tip3pfb_hole.xyz")
len(hole)

[5, 32, 80, 80, 80, 80, 80, 103, 103, 117, 117, 117, 127, 127, 127, 127, 127, 135, 155, 155, 155, 155, 179, 179, 225, 225, 225, 225, 225, 231, 231]


1106

In [14]:
tempo.save("1.xyz")

In [15]:
!cat "1.xyz" "tip3pfb_hole.xyz" > test.xyz